### Load the data

In [1]:
from sklearn.datasets import load_breast_cancer

breast_cancer = load_breast_cancer()

### Create dataframe from the data just to visualize it

In [2]:
import pandas as pd
df = pd.DataFrame(data=breast_cancer['data'], columns=breast_cancer['feature_names'])

df.insert(loc=0, column="present", value=breast_cancer['target'].tolist())

df.head()

,present,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,0,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,0,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,0,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
from sklearn.model_selection import train_test_split

x = breast_cancer['data']
y = breast_cancer['target']

x_train, x_test, y_train, y_test = train_test_split(
    x, y, random_state=0, stratify=y)

In [4]:
from math import sqrt, floor

n = df.shape[0]
k = floor(sqrt(n))
k

23

In [5]:
def euclidean_dist(point1, point2):
    if len(point1) != len(point2):
        raise Exception(
            f'Given point with different dimensions: {len(point1)}, {len(point2)}')

    sum = 0
    for feature1, feature2 in zip(point1, point2):
        squared_difference = (feature2 - feature1) ** 2
        sum += squared_difference

    return sqrt(sum)

In [6]:
def calc_dists(x_train, test_point):
    dists = []
    for train_point in x_train:
        dist = euclidean_dist(train_point, test_point)
        dists.append(dist)
    
    return dists

In [7]:
def get_neighbor_labels(dists, y_train, n_neighbors):
    dists_df = pd.DataFrame(data=dists, columns=['dist'])
    dists_df_sorted = dists_df.sort_values(by=['dist'], axis=0)

    neighbors_indices = dists_df_sorted[:n_neighbors].index.values
    neighbors_labels = [y_train[neighbor_index]
                        for neighbor_index in neighbors_indices]
    return neighbors_labels

In [8]:
from collections import Counter
def get_pred(neighbor_labels):
    counter = Counter(neighbor_labels)
    pred = counter.most_common(1)[0][0]
    return pred

In [9]:
def get_score(y_test, y_pred):
    correct_predictions = 0
    for class_test, class_pred in zip(y_test, y_pred):
        if class_test == class_pred:
            correct_predictions += 1
    return correct_predictions / len(y_test)

### Without Normalization

In [10]:
y_pred = []
for test_point in x_test:
    dists = calc_dists(x_train, test_point)
    neighbor_labels = get_neighbor_labels(dists, y_train, k)
    pred = get_pred(neighbor_labels)
    y_pred.append(pred)

score = get_score(y_test, y_pred)
score

0.9230769230769231

### With Normalization

In [11]:
x_mean = x.mean(axis=0)
x_std = x.std(axis=0)

x_train_norm = (x_train - x_mean) / x_std
x_test_norm = (x_test - x_mean) / x_std

In [12]:
y_pred = []
for test_point in x_test_norm:
    dists = calc_dists(x_train_norm, test_point)
    neighbor_labels = get_neighbor_labels(dists, y_train, k)
    pred = get_pred(neighbor_labels)
    y_pred.append(pred)

score = get_score(y_test, y_pred)
score

0.951048951048951